In [1]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import pandas as pd

In [2]:
df = pd.read_csv('./data/boston_rox.csv',sep = ",")

In [3]:
df["Date"] = pd.to_datetime(df["DateTime"])
df.set_index("Date", inplace=True)

In [4]:
df.drop(['DateTime'], axis=1, inplace=True)
df

,value
Date,
2022-11-24 03:00:00,9.0
2022-10-26 10:00:00,3.0
2022-11-02 14:00:00,9.0
2022-10-13 19:00:00,6.0
2023-01-12 23:00:00,7.0
...,...
2022-11-11 07:00:00,6.0
2022-11-02 19:00:00,5.0
2022-10-13 00:00:00,10.0


In [5]:
df_sorted = df.sort_values(by='Date')


In [6]:
df = df_sorted
df

,value
Date,
2022-10-01 00:00:00,8.0
2022-10-01 01:00:00,9.0
2022-10-01 02:00:00,8.0
2022-10-01 03:00:00,8.0
2022-10-01 04:00:00,8.0
...,...
2023-03-31 19:00:00,7.0
2023-03-31 20:00:00,10.0
2023-03-31 21:00:00,8.0


In [7]:
# Define the number of previous time steps to use for each prediction
n_steps = 24

# Split the data into training and testing sets
train_size = int(len(df) * 0.8)
train_data = df.iloc[:train_size]
test_data = df.iloc[train_size:]


In [8]:
train_data

,value
Date,
2022-10-01 00:00:00,8.0
2022-10-01 01:00:00,9.0
2022-10-01 02:00:00,8.0
2022-10-01 03:00:00,8.0
2022-10-01 04:00:00,8.0
...,...
2023-02-23 18:00:00,4.0
2023-02-23 19:00:00,4.0
2023-02-23 20:00:00,4.0


In [44]:
## Should only normalize training data not test data since we assume test data as completely new data. Since in real time we are

scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled = scaler.transform(test_data)

In [45]:
train_scaled

array([[0.33628319],
       [0.38053097],
       [0.33628319],
       ...,
       [0.15929204],
       [0.20353982],
       [0.24778761]])

In [46]:
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data) - n_steps):
        X.append(data[i:i+n_steps])
        y.append(data[i+n_steps])
    return np.array(X), np.array(y)

X_train, y_train = create_sequences(train_scaled, n_steps)
X_test, y_test = create_sequences(test_scaled, n_steps)

# Define the LSTM model architecture
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(n_steps, 1)))
model.add(LSTM(units=50))
model.add(Dense(units=1))


In [49]:
model.compile(optimizer='adam', loss='mean_squared_error')

In [47]:
X_train

array([[[0.33628319],
        [0.38053097],
        [0.33628319],
        ...,
        [0.24778761],
        [0.24778761],
        [0.24778761]],

       [[0.38053097],
        [0.33628319],
        [0.33628319],
        ...,
        [0.24778761],
        [0.24778761],
        [0.24778761]],

       [[0.33628319],
        [0.33628319],
        [0.33628319],
        ...,
        [0.24778761],
        [0.24778761],
        [0.24778761]],

       ...,

       [[0.24778761],
        [0.24778761],
        [0.24778761],
        ...,
        [0.15929204],
        [0.15929204],
        [0.15929204]],

       [[0.24778761],
        [0.24778761],
        [0.24778761],
        ...,
        [0.15929204],
        [0.15929204],
        [0.15929204]],

       [[0.24778761],
        [0.24778761],
        [0.20353982],
        ...,
        [0.15929204],
        [0.15929204],
        [0.20353982]]])

In [50]:
model.fit(X_train, y_train, epochs=100, batch_size=64)

Epoch 1/100
54/54 [==============================] - 4s 14ms/step - loss: 0.0186
Epoch 2/100
54/54 [==============================] - 1s 15ms/step - loss: 0.0099
Epoch 3/100
54/54 [==============================] - 1s 15ms/step - loss: 0.0079
Epoch 4/100
54/54 [==============================] - 1s 14ms/step - loss: 0.0065
Epoch 5/100
54/54 [==============================] - 1s 15ms/step - loss: 0.0055
Epoch 6/100
54/54 [==============================] - 1s 15ms/step - loss: 0.0046
Epoch 7/100
54/54 [==============================] - 1s 15ms/step - loss: 0.0040
Epoch 8/100
54/54 [==============================] - 1s 15ms/step - loss: 0.0037
Epoch 9/100
54/54 [==============================] - 1s 15ms/step - loss: 0.0035
Epoch 10/100
54/54 [==============================] - 1s 14ms/step - loss: 0.0029
Epoch 11/100
54/54 [==============================] - 1s 16ms/step - loss: 0.0027
Epoch 12/100
54/54 [==============================] - 1s 15ms/step - loss: 0.0028
Epoch 13/100
54/54 [=====

In [68]:
X_test

array([[[0.2920354 ],
        [0.38053097],
        [0.46902655],
        ...,
        [0.15929204],
        [0.15929204],
        [0.15929204]],

       [[0.38053097],
        [0.46902655],
        [0.55752212],
        ...,
        [0.15929204],
        [0.15929204],
        [0.15929204]],

       [[0.46902655],
        [0.55752212],
        [0.55752212],
        ...,
        [0.15929204],
        [0.15929204],
        [0.11504425]],

       ...,

       [[0.15929204],
        [0.15929204],
        [0.15929204],
        ...,
        [0.2920354 ],
        [0.2920354 ],
        [0.42477876]],

       [[0.15929204],
        [0.15929204],
        [0.15929204],
        ...,
        [0.2920354 ],
        [0.42477876],
        [0.33628319]],

       [[0.15929204],
        [0.15929204],
        [0.15929204],
        ...,
        [0.42477876],
        [0.33628319],
        [0.33628319]]])

In [53]:
y_pred = model.predict(X_test)

27/27 [==============================] - 0s 4ms/step


In [67]:
X_test.size

20088

In [54]:
y_pred.shape

(837, 1)

In [55]:
y_pred = scaler.inverse_transform(y_pred)
y_test = scaler.inverse_transform(y_test)

In [66]:
y_pred.size

837

In [69]:
y_pred24 = model.predict(X_test[-24:])
y_transform = scaler.inverse_transform(y_pred24)

1/1 [==============================] - 0s 18ms/step


In [70]:
y_transform 

array([[ 4.496511 ],
       [ 4.490515 ],
       [ 4.4826455],
       [ 4.47075  ],
       [ 5.4812226],
       [ 4.4583116],
       [ 4.387467 ],
       [ 4.4521413],
       [ 4.4231443],
       [ 4.4122987],
       [ 5.4535985],
       [ 5.47612  ],
       [ 5.415327 ],
       [ 5.4528255],
       [ 5.456668 ],
       [ 5.441556 ],
       [ 5.4352803],
       [ 6.4764194],
       [ 7.552893 ],
       [ 7.457464 ],
       [ 7.4093766],
       [10.652549 ],
       [ 8.417572 ],
       [ 8.156886 ]], dtype=float32)

y_pred.size

In [57]:
rmse = np.sqrt(np.mean((y_pred - y_test)**2))
print('RMSE:', rmse)

RMSE: 1.1477066460367897


In [58]:
last_24_hours = df.tail(24)[['value']].values
X_new = last_24_hours.reshape(1, 24, 1)

In [62]:
X_new.shape

(1, 24, 1)

In [21]:
y_new = model.predict(X_new)


1/1 [==============================] - 0s 24ms/step


In [59]:
import pandas as pd
import datetime
#Create dataframe with 24 rows and 0 columns

date_time_index = pd.date_range(start='00:00:00', end='23:00:00', freq='1H')
final = pd.DataFrame(index=date_time_index, columns=['value'])
final.iloc[:,0] = y_new

In [60]:
final

,value
2023-04-22 00:00:00,3.440367
2023-04-22 01:00:00,3.440367
2023-04-22 02:00:00,3.440367
2023-04-22 03:00:00,3.440367
2023-04-22 04:00:00,3.440367
2023-04-22 05:00:00,3.440367
2023-04-22 06:00:00,3.440367
2023-04-22 07:00:00,3.440367
2023-04-22 08:00:00,3.440367
2023-04-22 09:00:00,3.440367


In [63]:
type(y_new)

numpy.ndarray

In [64]:
print(f"Predicted PM2.5 concentration for the next 24 hours: {y_new[0, 0]:.2f}")


Predicted PM2.5 concentration for the next 24 hours: 3.44


In [23]:
df

,value
Date,
2022-10-01 00:00:00,8.0
2022-10-01 01:00:00,9.0
2022-10-01 02:00:00,8.0
2022-10-01 03:00:00,8.0
2022-10-01 04:00:00,8.0
...,...
2023-03-31 19:00:00,7.0
2023-03-31 20:00:00,10.0
2023-03-31 21:00:00,8.0


In [24]:
y_new

array([[3.4403672]], dtype=float32)

In [44]:
from sklearn.metrics import accuracy_score
acs = accuracy_score(y_pred.astype(int),y_test.astype(int))
print(acs)

0.4062126642771804


In [34]:
y_test.size

837

In [38]:
check  = pd.DataFrame()
check['value'] = pd.Series(y_test.flatten())
check['prediction'] = pd.Series(y_pred.flatten())
check

,value,prediction
0,4.0,4.292309
1,3.0,4.304964
2,4.0,3.346478
3,4.0,4.285405
4,4.0,4.377504
...,...,...
832,7.0,7.236440
833,10.0,7.183561
834,8.0,10.238671
835,8.0,8.238880


In [36]:
type(y_test)

numpy.ndarray

In [39]:
check.to_csv('out_lstm.csv')